In [ ]:
!pip install pymrmr

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

my_drive = '/content/gdrive/My Drive/'
import sys
sys.path.append(my_drive)

In [ ]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sklearn.metrics as metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
import statistics
from sklearn.metrics import matthews_corrcoef
import pymrmr
import pickle
import joblib
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRFClassifier
import csv

In [ ]:
data_dir = '/content/gdrive/My Drive/Brats 2018/MICCAI_BraTS_2018_Data_Training/'

In [ ]:
hgg_clf = '/content/gdrive/My Drive/Brats 2018/MICCAI_BraTS_2018_Data_Training/HGG_testing_data.csv'
lgg_clf = '/content/gdrive/My Drive/Brats 2018/MICCAI_BraTS_2018_Data_Training/LGG_testing_data.csv'

In [ ]:
hgg1=pd.read_csv(data_dir + "HGG_data.csv")#Path to csv
lgg1=pd.read_csv(data_dir + "LGG_data.csv")

In [ ]:
lgg1["GRADE"]=0
hgg1["GRADE"]=1
hgg= hgg1.sample(frac=1, random_state=42).reset_index(drop=True)
lgg= lgg1.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
auc_lr=[]
spe_lr=[]
sen_lr=[]
mcc_lr=[]
acc_lr=[]
f1_lr=[]
pre_lr=[]
rec_lr=[]
pre_lrt=[]
rec_lrt=[]
auc_lrt=[]
spe_lrt=[]
mcc_lrt=[]
sen_lrt=[]
acc_lrt=[]
f1_lrt=[]

In [ ]:
master=pd.concat([lgg, hgg])
master= master.sample(frac=1, random_state=42).reset_index(drop=True)
master= pd.concat([master["GRADE"], master.drop("GRADE", axis=1)], axis=1)
Y=master["GRADE"]
master = (master - master.min()) / (master.max() - master.min())

In [ ]:
# print(type(master))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
# cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
# for train_index, test_index in cv.split(master, Y):
#     #print("Train Index: ", train_index, "\n")
#     #print("Test Index: ", test_index)

#     train, test = master.iloc[train_index], master.iloc[test_index]
#     Y_train, Y_test= Y.iloc[train_index], Y.iloc[test_index]
#     # features=pymrmr.mRMR(train, 'MIQ', 40)

#     print(type(train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
model_list = []
model_result_list=[]
model_predicted=[]

x_test_vals = []
y_test_vals = []

fet = []

cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
clf = RandomForestClassifier(random_state=0, n_estimators=200, criterion='gini', max_features="auto", min_samples_split=10, min_samples_leaf=1)
for train_index, test_index in cv.split(master, Y):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    train, test = master.iloc[train_index], master.iloc[test_index]
    Y_train, Y_test= Y.iloc[train_index], Y.iloc[test_index]
    features=pymrmr.mRMR(train, 'MIQ', 40)#no of features selected-40
    print("Best Features: ", features)

    dataset=master[features]
    fet.append(features)

    X_train=dataset.iloc[train_index]
    X_test=dataset.iloc[test_index]

    #clf = XGBRFClassifier(random_state=0, n_estimators=200, criterion='gini', max_features="auto", min_samples_split=10, min_samples_leaf=1).fit(X_train, Y_train)
    # clf = RandomForestClassifier(random_state=0, n_estimators=200, criterion='gini', max_features="auto", min_samples_split=10, min_samples_leaf=1).fit(X_train, Y_train)
    clf = clf.fit(X_train, Y_train)

    my_model = clf
    model_list.append(clf)

    pred = clf.predict(X_test)

    x_test_vals.append(X_test)
    y_test_vals.append(Y_test)

    model_predicted.append(pred)
    # cm = metrics.confusion_matrix(Y_test, pred)
    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    sen_l = (TP / float(TP + FN))
    spe_l = (TN / float(TN + FP))
    pre_l = precision_score(Y_test, pred)
    rec_l = recall_score(Y_test, pred)
    mcc_l = matthews_corrcoef(Y_test, pred)
    acc_l = accuracy_score(Y_test, pred)
    probs = clf.predict_proba(X_test)
    probs = probs[:, 1]
    auc_l = roc_auc_score(Y_test, probs)
    f1_l= f1_score(Y_test, pred)


    pred = clf.predict(X_train)

    # cm = metrics.confusion_matrix(Y_train, pred)
    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    sen_lt = (TP / float(TP + FN))
    spe_lt = (TN / float(TN + FP))
    pre_lt = precision_score(Y_train, pred)
    rec_lt = recall_score(Y_train, pred)
    mcc_lt = matthews_corrcoef(Y_train, pred)
    acc_lt = accuracy_score(Y_train, pred)
    probs = clf.predict_proba(X_train)
    probs = probs[:, 1]
    auc_lt = roc_auc_score(Y_train, probs)
    f1_lt = f1_score(Y_train, pred)


    auc_lr.append(auc_l)
    spe_lr.append(spe_l)
    sen_lr.append(sen_l)
    mcc_lr.append(mcc_l)
    acc_lr.append(acc_l)
    f1_lr.append(f1_l)
    rec_lr.append(rec_l)
    pre_lr.append(pre_l)

    pre_lrt.append(pre_lt)
    rec_lrt.append(rec_lt)
    auc_lrt.append(auc_lt)
    spe_lrt.append(spe_lt)
    sen_lrt.append(sen_lt)
    mcc_lrt.append(mcc_lt)
    acc_lrt.append(acc_lt)
    f1_lrt.append(f1_lt)

    data = { "phase":["Testing", "Training"],
          "Precision" : [statistics.mean(pre_lr), statistics.mean(pre_lrt)],
          "Recall" : [statistics.mean(rec_lr), statistics.mean(rec_lrt)],
          "Accuracy" : [statistics.mean(acc_lr), statistics.mean(acc_lrt)],
          "AUC" : [statistics.mean(auc_lr), statistics.mean(auc_lrt)],
          "F1 Score" : [statistics.mean(f1_lr), statistics.mean(f1_lrt)]
        }
    model_result_list.append(data)

    model_list.append(clf)

# with open('saved_model_XGB', "wb") as f:
#   pickle.dumps(my_model, f)

#joblib.dump(my_model, data_dir + 'XGB_model.pkl')


print("TRAIN:")
print("PRE:", statistics.mean(pre_lrt))
print("REC:", statistics.mean(rec_lrt))
print("SPE:", statistics.mean(spe_lrt))
print("MCC:", statistics.mean(mcc_lrt))
print("SEN:", statistics.mean(sen_lrt))
print("ACC:", statistics.mean(acc_lrt))
print("AUC:", statistics.mean(auc_lrt))
print("F1:", statistics.mean(f1_lrt))

print("TEST:")
print("PRE:", statistics.mean(pre_lr))
print("REC:", statistics.mean(rec_lr))
print("SPE:", statistics.mean(spe_lr))
print("MCC:", statistics.mean(mcc_lr))
print("SEN:", statistics.mean(sen_lr))
print("ACC:", statistics.mean(acc_lr))
print("AUC:", statistics.mean(auc_lr))
print("F1:", statistics.mean(f1_lr))

Best Features:  ['original_firstorder_Maximum ROI 2', 'original_firstorder_Skewness ROI 2', 'original_glszm_SmallAreaEmphasis ROI 2', 'original_glrlm_LongRunEmphasis ROI 1', 'original_glrlm_GrayLevelNonUniformity ROI 1', 'original_firstorder_RootMeanSquared ROI 2', 'original_glszm_GrayLevelNonUniformityNormalized ROI 2', 'original_gldm_DependenceNonUniformityNormalized ROI 1', 'original_shape_Elongation ROI 3', 'original_ngtdm_Busyness ROI 1', 'original_shape_Sphericity ROI 2', 'original_glcm_MCC ROI 1', 'original_gldm_DependenceNonUniformity ROI 2', 'original_glcm_Idmn ROI 1', 'original_firstorder_TotalEnergy ROI 2', 'original_glrlm_RunPercentage ROI 1', 'original_glrlm_RunLengthNonUniformity ROI 1', 'original_firstorder_TotalEnergy ROI 3', 'original_firstorder_Kurtosis ROI 2', 'original_glszm_GrayLevelVariance ROI 3', 'original_ngtdm_Strength ROI 2', 'original_shape_Flatness ROI 3', 'original_ngtdm_Coarseness ROI 2', 'original_gldm_SmallDependenceLowGrayLevelEmphasis ROI 2', 'origina

In [ ]:
fet[4]

np.savetxt(data_dir + "Features.csv",
           fet[4],
           delimiter =", ",
           fmt ='% s')

In [ ]:
#model_result_list[4]
#my_model = model_list[4]

joblib.dump(clf, data_dir + 'RFC_model_01.pkl')

['/content/gdrive/My Drive/Brats 2018/MICCAI_BraTS_2018_Data_Training/RFC_model_01.pkl']

In [ ]:
data

{'AUC': [0.9495496930479509, 0.9962451465108524],
 'Accuracy': [0.9032205513784461, 0.9745553779579743],
 'F1 Score': [0.9355266586693146, 0.9829337646854016],
 'Precision': [0.9202595008176403, 0.9739597466077414],
 'Recall': [0.9522415795586527, 0.9921000855431993],
 'phase': ['Testing', 'Training']}

In [ ]:
metrics = pd.DataFrame.from_dict(data)
metrics.to_csv("/content/gdrive/My Drive/Brats 2018/MICCAI_BraTS_2018_Data_Training/metrics.csv")

In [ ]:
metrics

,phase,Precision,Recall,Accuracy,AUC,F1 Score
0,Testing,0.92026,0.952242,0.903221,0.949550,0.935527
1,Training,0.97396,0.992100,0.974555,0.996245,0.982934


In [ ]:
a = '/content/gdrive/My Drive/Brats 2018/MICCAI_BraTS_2018_Data_Training/RFC_model_01.pkl'

fileobj = open(a, 'rb')

b = pickle.load(fileobj)

print(b)

['original_firstorder_Maximum ROI 1' 'original_firstorder_Range ROI 2'
 'original_glszm_ZoneVariance ROI 1'
 'original_glszm_SmallAreaEmphasis ROI 1'
 'original_shape_MeshVolume ROI 2' 'original_shape_MinorAxisLength ROI 2'
 'original_shape_Sphericity ROI 2' 'original_glrlm_RunEntropy ROI 1'
 'original_glrlm_LongRunEmphasis ROI 1'
 'original_glszm_LargeAreaHighGrayLevelEmphasis ROI 1'
 'original_firstorder_Mean ROI 3'
 'original_glszm_LowGrayLevelZoneEmphasis ROI 2'
 'original_glcm_Imc2 ROI 1'
 'original_gldm_DependenceNonUniformityNormalized ROI 3'
 'original_glcm_MCC ROI 1' 'original_glszm_ZonePercentage ROI 1'
 'original_firstorder_TotalEnergy ROI 3'
 'original_firstorder_RootMeanSquared ROI 3'
 'original_glrlm_RunLengthNonUniformity ROI 1'
 'original_glszm_LargeAreaEmphasis ROI 2'
 'original_shape_Maximum3DDiameter ROI 3'
 'original_ngtdm_Coarseness ROI 3' 'original_shape_MinorAxisLength ROI 3'
 'original_glrlm_ShortRunLowGrayLevelEmphasis ROI 2'
 'original_shape_Maximum2DDiameterR